## Simulating ISING MODEL with metropolis update

In [ ]:
import numpy as np
DTYPE = np.float32

from numba import njit
import matplotlib.pyplot as plt
import pandas as pd
import os
from tqdm import tqdm

In [ ]:
"""
T: Temperature
L: Lattice Size
J: Interaction Strength
h: External Field
"""

@njit
def calculate_energy(spins, L, J, h):
    energy = 0
    for i in range(L):
        for j in range(L):
            neighbor_sum = spins[(i+1)%L, j] + spins[(i-1)%L, j] + spins[i, (j+1)%L] + spins[i, (j-1)%L]
            energy += -J*spins[i, j]*neighbor_sum
            
            energy += -h*spins[i, j]
    return energy


# Function to calculate the magnetization of the system
def calculate_magnetization(spins):
    return np.sum(spins)

# Metropolis
@njit
def monte_carlo_step(spins, L, T, J, h):
    for _ in range(L * L):  # One Monte Carlo step: attempt to flip L*L spins
        # Randomly select a spin
        i, j = np.random.randint(0, L, size=2)
        
        # Calculate the change in energy if the spin is flipped
        neighbor_sum = spins[(i+1)%L, j] + spins[(i-1)%L, j] + spins[i, (j+1)%L] + spins[i, (j-1)%L]
        delta_E = 2*J* spins[i, j]*neighbor_sum + 2*h*spins[i, j]
        
        # Metropolis acceptance criterion
        if delta_E < 0 or np.random.rand() < np.exp(-delta_E / T):
            spins[i, j] *= -1  


In [ ]:
L = int(16)
T = DTYPE(np.linspace(1, 20, 50)) # At even lower temperature, equilibriation time difficult to find because of domain growth
J = DTYPE(1)
hs =  DTYPE(np.linspace(0.1, 5, 25))

mc_steps = int(10000)
eq_steps = int(mc_steps/2)

In [ ]:
# Initial configuration
spins = np.random.choice([-1, 1], size = (L, L))
fig, ax = plt.subplots(figsize=(6, 6))
plt.imshow(spins)
ax.set_xticks(np.arange(0, L, L-1));
ax.set_yticks(np.arange(0, L, L-1));

In [ ]:
for h in hs:
    magnetizations = []
    susceptibilities = []
    specific_heats = []

    for temp in tqdm(T):
        spins = np.random.choice([-1, 1], size = (L, L))
        M_list = [] # Stores magnetisation after every step
        E_list = [] # Stores energy after every step
        
        for step in range(mc_steps):
            # Uncomment to save the evolution of the system
            # fig, ax = plt.subplots(figsize=(6, 6))
            # plt.imshow(spins)
            # ax.set_xticks(np.arange(0, L, L-1));
            # ax.set_yticks(np.arange(0, L, L-1));
            # title = f"L={L}" + "\n" + f"J={J:.3f}, h={h:.3f}, T={temp:.3f}" + "\n" + f"step {step}"
            # fig.suptitle(title)
        
            # output_filepath = f"data/L-{L}/J-{J:.3f}/h-{h:.3f}/T-{temp:.3f}/lattice_evolution"
            # output_filename = f"step{step}.png"
            # if not os.path.exists(output_filepath):
            #     os.makedirs(output_filepath)
        
            # file = os.path.join(output_filepath, output_filename)
            # plt.savefig(file, dpi = 100)
            # plt.close()
            
            if step > eq_steps:
                M = np.abs(calculate_magnetization(spins)) / (L * L)
                E = calculate_energy(spins, L, J, h)
                M_list.append(M)
                E_list.append(E)
            
            monte_carlo_step(spins, L, temp, J, h)
                
        # Compute averages
        M_avg = np.mean(M_list)
        M2_avg = np.mean(np.square(M_list))
        E_avg = np.mean(E_list)
        E2_avg = np.mean(np.square(E_list))
            
        magnetizations.append(M_avg)
        susceptibilities.append((M2_avg - M_avg**2)/temp)
        specific_heats.append((E2_avg - E_avg**2)/(temp**2))
    
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    
    # Magnetization
    ax[0].plot(T, magnetizations, 'o-')
    ax[0].set_xlabel("Temperature (T)")
    ax[0].set_ylabel("Magnetization (M)")
    ax[0].set_title("Magnetization vs. Temperature")
    
    # Susceptibility
    ax[1].plot(T, susceptibilities, 'o-')
    ax[1].set_xlabel("Temperature (T)")
    ax[1].set_ylabel("Susceptibility (χ)")
    ax[1].set_title("Susceptibility vs. Temperature")
    
    # Specific Heat
    ax[2].plot(T, specific_heats, 'o-')
    ax[2].set_xlabel("Temperature (T)")
    ax[2].set_ylabel("Specific Heat (C)")
    ax[2].set_title("Specific Heat vs. Temperature")
    
    title = f"L={L}" + "\n" + f"J={J:.3f}, h={h:.3f}"
    fig.suptitle(title)
        
    plt.tight_layout()
    output_filepath = f"data/2d/L-{L}/J-{J:.3f}/"
    output_filename = f"h-{h:.3f}.png"
    if not os.path.exists(output_filepath):
        os.makedirs(output_filepath)
    
    file = os.path.join(output_filepath, output_filename)
    plt.savefig(file, dpi = 100)
    plt.close()